In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
# Load your data
# df = pd.read_excel("your_file.xlsx")
# Simulated structure based on your screenshot:
df = pd.DataFrame({
    'behavior_extendedLeg': [...],  # video-labeled binary list
    'samples_IMU': [...],           # index of IMU samples
    'left_Knee_Angle': [...]        # corresponding IMU angle
})

In [ ]:
# === STEP 1: Align video labels (30 fps) with IMU samples (52 Hz) ===

# Video FPS and IMU rate
fps_video = 30
imu_rate = 52

# Compute time arrays
num_imu = df['samples_IMU'].max() + 1
imu_time = np.arange(num_imu) / imu_rate
video_time = np.arange(len(df['behavior_extendedLeg'])) / fps_video

# Interpolate video labels to match IMU timestamps
label_interp = np.interp(imu_time, video_time, df['behavior_extendedLeg'])

# Binary classification: apply 0.5 threshold
label_binary = (label_interp > 0.5).astype(int)

# Get angle data
knee_angle = df['left_Knee_Angle'].values

# Match sizes if needed
X = knee_angle[:len(label_binary)].reshape(-1, 1)
y = label_binary[:len(knee_angle)]

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Prediction
y_pred_cont = model.predict(X_test)
# convert binary
y_pred = (y_pred_cont > 0.5).astype(int)

In [ ]:
# Genaigkeit
acc = accuracy_score(y_test, y_pred)
print(f"Classification accuracy: {acc:.2f}")

In [ ]:
plt.scatter(X_test, y_test, color='blue', label='True Label')
plt.scatter(X_test, y_pred, color='red', marker='x', label='Predicted')
plt.xlabel("Left Knee Angle")
plt.ylabel("Extended Leg (0/1)")
plt.title("Prediction vs Ground Truth")
plt.legend()
plt.grid()
plt.show()